In [1]:
import pathlib
import os
from yacs.config import CfgNode
from src.model.perception.evaluation import get_ground_truth
from src.model.perception.model_wrapper import ModelWrapper
from src.model.perception.perception_pipeline_config import get_perception_cfg
from src.data.MaskRCNNEvaluationDataset import MaskRCNNEvaluationDataset, eval_collate_fn
from pathlib import Path
from torch.utils.data import DataLoader
import numpy as np


In [2]:
if pathlib.PurePath(os.getcwd()).name == "notebooks":
    print(pathlib.PurePath(os.getcwd()).name)
    os.chdir("..")

notebooks


In [3]:
perception_cfg = get_perception_cfg()
perception_cfg.DATA_PATHS.TRAJECTORIES_DIR = str(Path(
        perception_cfg.DATA_PATHS.INTERIM_DATA_DIR, "trajectories", "23-06-11,16:09:05"))

eval_dataset = MaskRCNNEvaluationDataset(perception_cfg.DATA_PATHS, perception_cfg.DATA_GENERATOR.SPLIT, 0)
eval_dataloader = DataLoader(eval_dataset,
                                batch_size=8,
                                shuffle=False,
                                num_workers=perception_cfg.TRAINING.NUM_WORKERS,
                                collate_fn=eval_collate_fn)



In [18]:
model_config = CfgNode()
model_config.USE_INITIAL_TRANSFORMS = True
model_config.SCORE_THRESHOLD = 0.5
model_config.MASK_THRESHOLD = 0.5
model_config.TRAINABLE_BACKBONE_LAYERS = 0
model = ModelWrapper(model_config, weights = "models/MaskRCNN/model_ep4.pth")
model.cuda()

In [21]:
from tqdm import tqdm
metrics = []
weights = []

for i, (images, semantics, semantic_info_paths) in enumerate(tqdm(eval_dataloader)):
    ground_truths = [get_ground_truth(semantics[i], semantic_info_paths[i]) for i in range(len(images))]
    mask = np.array([len(ground_truth['boxes']) for ground_truth in ground_truths]) > 0
    if np.sum(mask) == 0:
        continue
    metrics.append(model.get_metrics(images[mask], [ground_truth for i, ground_truth in enumerate(ground_truths) if mask[i]]))
    weights.append(np.sum(mask))

100%|██████████| 75/75 [00:23<00:00,  3.13it/s]


In [9]:
# Epoch 0: 0.18502388905171316
weights_np = np.array(weights)
metrics_np = np.array(metrics)
np.sum(weights_np*metrics_np)/np.sum(weights_np)

0.18502388905171316

In [13]:
# Epoch 1: 0.21526870020052308
weights_np = np.array(weights)
metrics_np = np.array(metrics)
np.sum(weights_np*metrics_np)/np.sum(weights_np)

0.21526870020052308

In [17]:
# Epoch 2: 0.21901360952276788
weights_np = np.array(weights)
metrics_np = np.array(metrics)
np.sum(weights_np*metrics_np)/np.sum(weights_np)

0.21901360952276788

In [6]:
# Epoch 3: 0.2544390270088473
weights_np = np.array(weights)
metrics_np = np.array(metrics)
np.sum(weights_np*metrics_np)/np.sum(weights_np)

0.2544390270088473